In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
directory = '/content/drive/My Drive/ActivityData'

In [4]:
samples_per_activity = 238.0

In [5]:
#remove empty lines from files
def save_to_dataframe(filepath):
  dataframe = pd.read_csv(filepath)
  dataframe = dataframe.dropna()
  return dataframe

In [6]:
jogging_df    = save_to_dataframe(directory+'/Jogging.csv')
walking_df    = save_to_dataframe(directory+'/Walking.csv')
stationary_df = save_to_dataframe(directory+'/Stationary.csv')
upstairs_df   = save_to_dataframe(directory+'/Upstairs.csv')
downstairs_df = save_to_dataframe(directory+'/Downstairs.csv')

jogging_test_df    = save_to_dataframe(directory+'/test/jogging_test.csv')
walking_test_df    = save_to_dataframe(directory+'/test/walking_test.csv')
stationary_test_df = save_to_dataframe(directory+'/test/stationary_test.csv')
upstairs_test_df   = save_to_dataframe(directory+'/test/upstairs_test.csv')
downstairs_test_df = save_to_dataframe(directory+'/test/downstairs_test.csv')

In [7]:
#get value to reshape to make 3d numpy array
reshape_jogging =int(jogging_df.values.shape[0]      / (samples_per_activity))
reshape_walking = int(walking_df.values.shape[0]     / (samples_per_activity))
reshape_stationary = int(stationary_df.values.shape[0]/ (samples_per_activity))
reshape_upstairs = int(upstairs_df.values.shape[0]   / (samples_per_activity))
reshape_downstairs= int(downstairs_df.values.shape[0]/ (samples_per_activity))

reshape_jogging_test =int(jogging_test_df.values.shape[0]      / (samples_per_activity))
reshape_walking_test = int(walking_test_df.values.shape[0]     / (samples_per_activity))
reshape_stationary_test = int(stationary_test_df.values.shape[0]/ (samples_per_activity))
reshape_upstairs_test = int(upstairs_test_df.values.shape[0]   / (samples_per_activity))
reshape_downstairs_test = int(downstairs_test_df.values.shape[0]/ (samples_per_activity))

In [8]:
#reshape into 3d numpy array with previously found values
jogging = jogging_df.values.reshape((reshape_jogging,238,6))
walking = walking_df.values.reshape((reshape_walking,238,6))
stationary = stationary_df.values.reshape((reshape_stationary,238,6))
upstairs = upstairs_df.values.reshape((reshape_upstairs,238,6))
downstairs = downstairs_df.values.reshape((reshape_downstairs,238,6))

jogging_t = jogging_test_df.values.reshape((reshape_jogging_test,238,6))
walking_t = walking_test_df.values.reshape((reshape_walking_test,238,6))
stationary_t = stationary_test_df.values.reshape((reshape_stationary_test,238,6))
upstairs_t = upstairs_test_df.values.reshape((reshape_upstairs_test,238,6))
downstairs_t = downstairs_test_df.values.reshape((reshape_downstairs_test,238,6))

In [9]:
#print how many samples for each activity and shape
print("Train:")
print(jogging.shape[0],'samples of Jogging ',jogging.shape)
print(walking.shape[0],'samples of Walking ',walking.shape)
print(stationary.shape[0],'samples of Stationary',stationary.shape)
print(upstairs.shape[0],'samples of Upstairs ',upstairs.shape)
print(downstairs.shape[0],'samples of Downstairs ',downstairs.shape)

print("Test:")
print(jogging_t.shape[0],'samples of Jogging ',jogging_t.shape)
print(walking_t.shape[0],'samples of Walking ',walking_t.shape)
print(stationary_t.shape[0],'samples of Stationary',stationary_t.shape)
print(upstairs_t.shape[0],'samples of Upstairs ',upstairs_t.shape)
print(downstairs_t.shape[0],'samples of Downstairs ',downstairs_t.shape)

Train:
42 samples of Jogging  (42, 238, 6)
91 samples of Walking  (91, 238, 6)
47 samples of Stationary (47, 238, 6)
49 samples of Upstairs  (49, 238, 6)
39 samples of Downstairs  (39, 238, 6)
Test:
10 samples of Jogging  (10, 238, 6)
10 samples of Walking  (10, 238, 6)
10 samples of Stationary (10, 238, 6)
10 samples of Upstairs  (10, 238, 6)
10 samples of Downstairs  (10, 238, 6)


In [10]:
#create trainy that corresponds to each array and combine into one array, use one hot encoding
jogging_y    = np.zeros(jogging.shape[0])
walking_y    = np.ones(walking.shape[0])
stationary_y = np.full((stationary.shape[0]),2)
upstairs_y   = np.full((upstairs.shape[0]),3)
downstairs_y = np.full((downstairs.shape[0]),4)
trainy = np.concatenate((jogging_y,walking_y,stationary_y,upstairs_y,
                         downstairs_y))
trainy = tf.keras.utils.to_categorical(trainy)

#create testy the same way
jogging_ytest = np.zeros(jogging_t.shape[0])
walking_ytest    = np.ones(walking_t.shape[0])
stationary_ytest = np.full((stationary_t.shape[0]),2)
upstairs_ytest   = np.full((upstairs_t.shape[0]),3)
downstairs_ytest = np.full((downstairs_t.shape[0]),4)
testy = np.concatenate((jogging_ytest,walking_ytest,stationary_ytest,upstairs_ytest,
                         downstairs_ytest))
testy = tf.keras.utils.to_categorical(testy)

In [11]:
trainX = np.concatenate((jogging,walking,stationary,upstairs,downstairs))
testX = np.concatenate((jogging_t,walking_t,stationary_t,upstairs_t,downstairs_t))

In [12]:
print("Train: ")
print(trainX.shape)
print(trainy.shape)

print("Test: ")
print(testX.shape)
print(testy.shape)

Train: 
(268, 238, 6)
(268, 5)
Test: 
(50, 238, 6)
(50, 5)


In [13]:
#build the model LSTM
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(30,input_shape=(238,6)))
model.add(tf.keras.layers.Dense(5,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(trainX,trainy,epochs=100)

Epoch 1/100
9/9 [==============================] - 1s 65ms/step - loss: 1.7596 - accuracy: 0.2201
Epoch 2/100
9/9 [==============================] - 1s 68ms/step - loss: 1.5398 - accuracy: 0.3881
Epoch 3/100
9/9 [==============================] - 1s 67ms/step - loss: 1.3643 - accuracy: 0.4888
Epoch 4/100
9/9 [==============================] - 1s 74ms/step - loss: 1.2344 - accuracy: 0.5746
Epoch 5/100
9/9 [==============================] - 1s 66ms/step - loss: 1.1201 - accuracy: 0.6567
Epoch 6/100
9/9 [==============================] - 1s 66ms/step - loss: 1.0208 - accuracy: 0.7201
Epoch 7/100
9/9 [==============================] - 1s 67ms/step - loss: 0.9352 - accuracy: 0.7649
Epoch 8/100
9/9 [==============================] - 1s 74ms/step - loss: 0.8561 - accuracy: 0.8097
Epoch 9/100
9/9 [==============================] - 1s 66ms/step - loss: 0.7923 - accuracy: 0.8246
Epoch 10/100
9/9 [==============================] - 1s 68ms/step - loss: 0.7344 - accuracy: 0.8433
Epoch 11/100
9/9 [=

In [ ]:
#run this cell if utilizing CNN-LSTM model
n_timesteps,n_features,n_outputs = trainX.shape[1],trainX.shape[2], trainy[1]

#reshape data into time steps of sub-sequences
n_steps, n_length = 2, 119

trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
print(trainX.shape)
print(testX.shape)

(268, 2, 119, 6)
(50, 2, 119, 6)


In [ ]:
#model CNN-LSTM
model = tf.keras.Sequential()
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu')))
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dropout(0.5)))
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling1D(pool_size=2)))
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()))
model.add(tf.keras.layers.LSTM(100))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
history = model.fit(trainX,trainy, epochs=100)

In [ ]:
predictions = model.predict(testX)
print("predictions =\n", np.round(predictions, decimals=3))
print("actual =\n", testy)

predictions =
 [[0.969 0.025 0.001 0.002 0.002]
 [0.929 0.06  0.001 0.006 0.004]
 [0.229 0.733 0.013 0.004 0.021]
 [0.975 0.013 0.002 0.004 0.005]
 [0.991 0.005 0.001 0.001 0.001]
 [0.969 0.019 0.005 0.005 0.001]
 [0.991 0.002 0.    0.003 0.003]
 [0.57  0.367 0.013 0.006 0.043]
 [0.591 0.073 0.015 0.026 0.295]
 [0.383 0.161 0.115 0.021 0.321]
 [0.01  0.972 0.005 0.004 0.009]
 [0.047 0.852 0.009 0.06  0.031]
 [0.011 0.962 0.001 0.006 0.02 ]
 [0.008 0.986 0.001 0.001 0.003]
 [0.115 0.85  0.003 0.005 0.028]
 [0.01  0.957 0.005 0.015 0.013]
 [0.012 0.974 0.002 0.006 0.005]
 [0.006 0.986 0.003 0.002 0.003]
 [0.008 0.972 0.001 0.004 0.015]
 [0.238 0.724 0.004 0.007 0.026]
 [0.    0.    0.998 0.    0.001]
 [0.    0.    0.998 0.    0.001]
 [0.    0.    0.998 0.    0.001]
 [0.    0.    0.998 0.    0.001]
 [0.    0.    0.998 0.    0.001]
 [0.    0.    0.998 0.    0.001]
 [0.    0.    0.998 0.    0.001]
 [0.    0.    0.998 0.    0.001]
 [0.    0.    0.998 0.    0.001]
 [0.    0.    0.998 0.    0.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

open("gesture_model.tflite", "wb").write(tflite_model)

import os
basic_model_size = os.path.getsize("gesture_model.tflite")
print("Model is %d bytes" % basic_model_size)

In [ ]:
!echo "const unsigned char model[] __attribute__((aligned(4))) = {"  > /content/model.h
!cat gesture_model.tflite | xxd -i                                  >> /content/model.h
!echo "};"                                                          >> /content/model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")